In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

## Check table

In [11]:
%%sql
SELECT name, type
    FROM sqlite_master
    WHERE type IN ("table", "view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


## A query to find out which genres sell the most tracks in the USA
* USA Genre name, number of tracks 
* each invoice contains multiple invoice_line, each invoice_line refer to a track

In [77]:
%%sql 
WITH invoice_usa AS 
    (SELECT invoice_id FROM invoice i WHERE i.billing_country = 'USA'),
     track_information AS 
    (SELECT iu.invoice_id, g.name, il.invoice_line_id
        FROM track t
        INNER JOIN genre g ON t.genre_id = g.genre_id
        INNER JOIN invoice_line il ON il.track_id = t.track_id
        INNER JOIN invoice_usa iu ON iu.invoice_id = il.invoice_id)

SELECT ti.name AS genre_name, COUNT(ti.name) AS number_sold, 
        ROUND(CAST(COUNT(ti.name) AS Float) / (SELECT COUNT(*) FROM track_information) * 100,1)
        AS sold_percentage
FROM track_information ti
GROUP BY 1
ORDER BY 2 DESC

Done.


genre_name,number_sold,sold_percentage
Rock,561,53.4
Alternative & Punk,130,12.4
Metal,124,11.8
R&B/Soul,53,5.0
Blues,36,3.4
Alternative,35,3.3
Latin,22,2.1
Pop,22,2.1
Hip Hop/Rap,20,1.9
Jazz,14,1.3


* Based on the tracks sold in USA, 1.9% sold tracks belongs to Hip-Hop, Pop is 2.1% and Blues is 3.4, while Punks is 12.4%, so of those four artist, Red Tone which is Punk, Slim Jim Bites with genre Blues and Meteor and the Girls with genre Pop are recommended to be purchased since those three genres rank higher comparing to Hip-Hop.

## Analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.
* Write a query that finds the total dollar amount of sales assigned to each sales support agent within the company. Add aby extra attributes for the employee which are relevant to the analysis

In [89]:
%%sql
WITH customer_invoice AS
    (SELECT c.customer_id customer_id, c.support_rep_id, SUM(i.total) total
     FROM customer c
     LEFT JOIN invoice i ON c.customer_id = i.customer_id
     GROUP BY 1)
SELECT 
    e.first_name||" "||e.last_name agent_name,
    e.title,
    e.hire_date,
    e.country,
    SUM(total) total
FROM (SELECT * FROM employee WHERE title = 'Sales Support Agent') e
LEFT JOIN customer_invoice ci ON e.employee_id = ci.support_rep_id
GROUP BY employee_id
ORDER BY 5 DESC

Done.


agent_name,title,hire_date,country,total
Jane Peacock,Sales Support Agent,2017-04-01 00:00:00,Canada,1731.51
Margaret Park,Sales Support Agent,2017-05-03 00:00:00,Canada,1584.0
Steve Johnson,Sales Support Agent,2017-10-17 00:00:00,Canada,1393.92


* As common sense, the earlier the agent is hired, the more dollar amount of sales assigned to that agent.

## Analyze the sales data for customers from each different country.
* A query that collates data on purchases from different countries

In [117]:
%%sql
CREATE VIEW customer_invoice AS
    SELECT  customer_id, 
            SUM(total) customer_total, 
            COUNT(*) customer_order_number
     FROM invoice
     GROUP BY 1;
CREATE VIEW country_sales AS
    SELECT 
    c.country, 
    COUNT(ci.customer_id) customer_number,
    SUM(ci.customer_total) total_sales,
    AVG(ci.customer_total) avg_sale_per_customer,
    SUM(ci.customer_order_number) / COUNT(ci.customer_id) avg_order_value
FROM customer c
INNER JOIN customer_invoice ci ON c.customer_id = ci.customer_id
GROUP BY c.country
ORDER BY 3 DESC;

Done.
Done.


[]

In [123]:
%%sql
SELECT cs.country, 
    customer_number, 
    total_sales, 
    avg_sale_per_customer,
    avg_order_value
FROM 
    (
    SELECT 
        cs.*,
        CASE 
            WHEN cs.customer_number = 1 THEN 1
            ELSE 0
        END AS sort
        FROM country_sales cs) cs
ORDER BY sort ASC

Done.


country,customer_number,total_sales,avg_sale_per_customer,avg_order_value
USA,13,1040.49,80.03769230769231,10
Canada,8,535.59,66.94875,9
Brazil,5,427.67999999999995,85.53599999999999,12
France,5,389.07,77.814,10
Germany,4,334.62,83.655,10
Czech Republic,2,273.24,136.62,15
United Kingdom,3,245.51999999999998,81.83999999999999,9
Portugal,2,185.13,92.565,14
India,2,183.14999999999998,91.57499999999999,10
Ireland,1,114.83999999999997,114.83999999999997,13


## A query that categorizes each invoice as either an album purchase or not, and calculates the Number of invoices and Percentage of invoices
* Find out what percentage of purchases are individual tracks vs whole albums.
* Have to identify whether each invoice has all the tracks from an album.
* Get the list of tracks from an invoice and comparing it to the list of tracks from an album.
* The album to compare the purchase can be found by lookingup the album that one of the purchased tracks belongs to. One track belonging to a album is enough to find the album.
* Add a column to each invoice, to indicate it either an album purchase or not
* Calculate number and percentage of invoices belonging to each group

In [184]:
%%sql
WITH album_track AS
    (SELECT 
        a.album_id, t.track_id
     FROM album a
     LEFT JOIN track t ON a.album_id = t.album_id),
    invoice_track AS
    (SELECT
        invoice_id, track_id
     FROM invoice_line
     ORDER BY 1)
SELECT invoice_id,
    CASE
        WHEN (SELECT COUNT(track_id) FROM invoice_track it WHERE it.invoice_id = invoice_id) >10 THEN 1
        ELSE 0
        END as sort
FROM invoice
--SELECT track_id FROM invoice_track WHERE invoice_id = 
--SELECT track_id FROM album_track WHERE album_id = 


Done.


invoice_id,sort
16,1
77,1
149,1
153,1
182,1
184,1
223,1
270,1
296,1
442,1


In [140]:
%%sql
SELECT
        invoice_id, track_id
     FROM invoice_line
    ORDER BY 1

Done.


invoice_id,track_id
1,1158
1,1159
1,1160
1,1161
1,1162
1,1163
1,1164
1,1165
1,1166
1,1167


In [165]:
%%sql
SELECT e.*,
        CASE 
            WHEN e.first_name = 'And' THEN 1
            ELSE 0
            END AS aaa
FROM employee e

Done.


employee_id,last_name,first_name,title,reports_to,birthdate,hire_date,address,city,state,country,postal_code,phone,fax,email,aaa
1,Adams,Andrew,General Manager,None,1962-02-18 00:00:00,2016-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com,0
2,Edwards,Nancy,Sales Manager,1,1958-12-08 00:00:00,2016-05-01 00:00:00,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com,0
3,Peacock,Jane,Sales Support Agent,2,1973-08-29 00:00:00,2017-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com,0
4,Park,Margaret,Sales Support Agent,2,1947-09-19 00:00:00,2017-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com,0
5,Johnson,Steve,Sales Support Agent,2,1965-03-03 00:00:00,2017-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com,0
6,Mitchell,Michael,IT Manager,1,1973-07-01 00:00:00,2016-10-17 00:00:00,5827 Bowness Road NW,Calgary,AB,Canada,T3B 0C5,+1 (403) 246-9887,+1 (403) 246-9899,michael@chinookcorp.com,0
7,King,Robert,IT Staff,6,1970-05-29 00:00:00,2017-01-02 00:00:00,590 Columbia Boulevard West,Lethbridge,AB,Canada,T1K 5N8,+1 (403) 456-9986,+1 (403) 456-8485,robert@chinookcorp.com,0
8,Callahan,Laura,IT Staff,6,1968-01-09 00:00:00,2017-03-04 00:00:00,923 7 ST NW,Lethbridge,AB,Canada,T1H 1Y8,+1 (403) 467-3351,+1 (403) 467-8772,laura@chinookcorp.com,0
